# 모듈가져오기

In [1]:
# 기본 토치
import torch
# 레이어 구성
import torch.nn as nn
# 함수들, 평가
import torch.nn.functional as F
# 최적화
import torch.optim as optim

# 데이터 공급 -> 학습에 필요한 데이터 공급
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [2]:
# GPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
DEVICE

device(type='cuda')

# 데이터 준비 (수집, 준비, 전처리)

- 훈련시 사용하는 데이터 공급은 공급자를 통해서 제공
- FashionMNIST
    - h, w = 28,28
    - 정답 10개

In [3]:
# 배치사이즈
BATCH_SIZE = 64 # 설정값

In [4]:
# 훈련 데이터 공급자
train_loader = DataLoader(
    dataset     = datasets.FashionMNIST(
        root    = './data',  # 다운로드될 위치, 데이터를 읽어오는 위치, 저장된 위치
        train   = True,      # 이 데이터는 훈련용임
        download= True,      # 데이터가 해당 경로에 없으면 다운로드
        transform   = transforms.Compose([
            # 이미지(데이터)에 대한 전처리
            transforms.ToTensor(), # 데이터를 공급할대 텐서로 제공하겠다
            transforms.Normalize( 0.2, 0.3 ) # 평균, 표준편차
        ]),
    ),  # 패션 이미지 데이터
    batch_size  = BATCH_SIZE,
    shuffle     = True
)

100%|██████████| 26421880/26421880 [00:01<00:00, 13367333.48it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 271706.43it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5036697.92it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20741860.70it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [5]:
# 테스트 데이터 공급자
test_loader = DataLoader(
    dataset     = datasets.FashionMNIST(
        root    = './data',  # 다운로드될 위치, 데이터를 읽어오는 위치, 저장된 위치
        train   = False,      # 이 데이터는 테스트용임
        download= True,      # 데이터가 해당 경로에 없으면 다운로드
        transform   = transforms.Compose([
            # 이미지(데이터)에 대한 전처리
            transforms.ToTensor(), # 데이터를 공급할대 텐서로 제공하겠다
            transforms.Normalize( 0.2, 0.3 ) # 평균, 표준편차
        ]),
    ),  # 패션 이미지 데이터
    batch_size  = BATCH_SIZE,
    shuffle     = True
)

# 인공신경망 구축- 객체 지향 스타일

```
- 입력층
- 은닉층
    - 합성곱층 1f
    - 풀링층 1f
    - 합성곱층 2f :
    - 풀링층 2f :
    - flattern + 전결합층 :
    - 드롭아웃층
- 출력층
```

In [6]:
class Net(nn.Module):
    # 생성자
    def __init__(self):
        '''
            - 중요한 각 층(layer)을 구성(맴버 변수)
        '''
        super(Net, self).__init__()
        # 맴버 변수로 만들지 않은 층은 => 함수로 처리하겠다 (설정)
        # 합성곱층 2개
        self.conv1      = nn.Conv2d(1,  32, 5, 1, padding='same') # 합성곱 1f
        self.conv2      = nn.Conv2d(32, 64, 5, 1, padding='same') # 합성곱 2f
        # 과적합 방지층
        self.dropout    = nn.Dropout2d(0.1) # 10% 훈련 방해 비율
        # 전결합층
        # shape 설계가 된후 작성 가능
        # 28 -> 14 -> 7 : flatter =>? 7*7*64
        self.fc         = nn.Linear( 7*7*64, 1024 ) # 1024 수렴
        # 출력층
        self.output     = nn.Linear( 1024, 10)      # 10 수렴
        pass

    # 순전파신경망 - 인공신경망 구성 -
    # 함수 입력 => 입력층
    # 함수 출력 => 출력층
    def forward(self, x):
        # x : 입력층
        # 합성곱층(합성곱층, 풀링층) 이런식으로 묶어서 표현도 가능 -> feature map -> activation_map
        # 이미지 크기 28 -> 14
        # 객체를 함수처럼 사용 => 내부에 __call__() 구현되있다면 사용가능
        x = F.relu( F.max_pool2d( self.conv1(x), 2 )) # 1f (합성곱, 풀링) 완료
        # 2f, dropout을 부여한다면 (임의로 부여)
        # 이미지 크기 14 -> 7
        x = F.relu( F.max_pool2d( self.dropout(self.conv2(x)), 2 ))
        # 전결합층에 결합하기 위해 -> 사전에 flattern 처리(4D->2D)
        # x텐서 => 4D->2D 처리를 위해 view()로 처리
        x = x.view( -1, 64*7*7) # 데이터 1개당(이미지 1개당) => 7*7의 데이터가 64개 준비되어 있음
        # 전결합층
        x = self.dropout( F.relu( self.fc(x) ) )
        # 출력층
        # F.softmax() => 데이터를 원본 그대로 사용한다면 주로 사용
        # F.log_softmax() => 데이터가 정규화등으로 조정된다면 -> 분포 변경이 될경우 주로 사용
        return F.log_softmax( self.output( x ), dim=1) # 개별 옷에 대한 확률값

In [7]:
model = Net().to(DEVICE) # cpu|cuda를 설정
model

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (dropout): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=3136, out_features=1024, bias=True)
  (output): Linear(in_features=1024, out_features=10, bias=True)
)

# 학습에 대한 도구 세팅

- 케리스
    - 컴파일 옵션
        - 최적화도구, 손실함수, 평가지표
- 토치
    - 대동 소이함
        - 최적화도구 설정
        - 손실, 평가지표등(정확도)등은 실제 학습시 처리하는것으로 조정

In [8]:
model.parameters() # 학습을 통해서 최적화 해야할 가중치, 편향등

<generator object Module.parameters at 0x7e3f93794cf0>

In [9]:
# 최적화 도구
# 최적화 대상, 학습률, 가속도
opimzer = optim.SGD( model.parameters(), lr=0.01, momentum=0.5)

# 훈련( 학습 )

## 훈련용 함수

- 크로스엔트로피
    - 두 확률 분포 간의 차이를 측정하는 메트릭
    - 실제값, 예측값 사이의 차이 => 손실함수

In [10]:
def train( model, train_loader, opimzer, epoch, verbose=True):
    '''
        모델 학습 함수
        - model : 훈련시킬 모델
        - train_loader : 훈련용 데이터 공급용 공급자
        - opimzer : 모델의 파라미터를 최적화할 도구
        - epoch : 학습 세대 값
    '''
    # 1. 모델을 학습 모드로 전환
    model.train()
    # 2. 미니 배치(컨셉) 학습
    for idx, (data, target) in enumerate( train_loader ): # 학습순번,피처데이터, 정답데이터
        # 2-1. cpu or cuda 설정
        data, target = data.to(DEVICE), target.to(DEVICE)
        # 2-2. 최적화 도구 초기화 진행. 학습이 진행되면 도구가 조정이 된다 => 다음번 학습시 영향을 미침, 0으로 조정
        opimzer.zero_grad()
        # 2-3. 데이터를 모델에 주입 -> 출력층 (개별 옷에 대한 확률) : 순전파
        output = model( data )
        # 2-4. 실제값과 예측값 사이 차이 계산 => 크로스엔트로피 함수를 활용 => loss
        loss = F.cross_entropy( output, target )
        # 2-5. 오차역전파 진행 (y->x 이동하면서 가중치 업데이트) => 최적화 행위 => 가중치들은 기록
        loss.backward()
        # 2-6. 최적화된 파라미터를 모델에 반영 => 학습 행위 1회 종료
        opimzer.step()
        # 2-7. 로그 출력(필요시, 필요한 타이밍에 맞춰)
        if verbose and idx % 200 == 0:  # 200단위로 로그 출력
            # loss => 텐서 => 차원표시가 나옴[ [] ]<= 이런 표시를 제외한 실제값 출력 item()
            print( f'Epoch:{epoch}\t mini-batch:{idx}\t loss:{loss.item()}' )

        pass
    # 3. 체크 포인트 저장 -> 모델 덤프 (*.pt or *.pth)
    # 세대별 학습이 마무리되면 저장, 이전 모델보다 성능이 향상되면 저장(컨셉을 부여, 추가 구현)
    torch.save({
        'model'                 : 'train-model',
        'epoch'                 : epoch,
        'model_state_dict'      : model.state_dict(),
        'opimzer_state_dict'    : opimzer.state_dict(),
        # 필요시 추가 정보
        'desc'                  : f'train-model checkpoint-{epoch}'
    }, f'./model-checkpoint-{epoch}.pt')
    pass

## 테스트용 함수

In [11]:
def test( model, test_loader ):
    '''
        세대별(epoch) 학습결과를 기반 -> 모델을 이용하여 테스트 진행 -> 평균 손실, 정확도 점검
        - model : 현재까지 학습된 모델
        - test_loader : 테스트 데이터 공급용 공급자
    '''
    # 1. 테스트 결과를 담을 변수 초기화
    mean_loss, mean_acc = (0,0)
    # 2. 모델을 테스트 모드로 전환
    model.eval()
    # 3. 테스트 진행한 모든 내용은 기록하지 않는다 세팅 -> 모델 성능에 영향 X
    with torch.no_grad(): # 이후 진행되는 행위에 대한 계산, 기록 X, 테스트 데이터 모두 사용->테스트완료
        # 3-1 테스트 데이터 공급 테스트 진행
        for idx, (data, target) in enumerate( test_loader ):
            # 3-1-1 cpu or cuda 설정
            data, target = data.to(DEVICE), target.to(DEVICE)
            # 3-1-2 모델에 데이터 주입
            output = model( data )
            # 3-1-3 실제값, 예측값 차이를 계산 => 손실함수 => 누적해서 합산 => mean_loss
            # 데이터 64개에 대한 오차값 => 합산 => .... => 모든데이터에 대한 오차값 합산
            mean_loss += F.cross_entropy( output, target, reduction='sum' ).item()
            # 3-1-4 정확도 계산(텐서 연산) => 누적합산 => mean_acc
            # 출력층(텐서) -> 각 10개의 옷의 확률값 보관(...softmax)-> 대상 텐서들중에서 가장 높은값을가진 텐서의 인덱스 정보 획득
            # output => 2D, (데이터의수, 10)
            y_pred = output.max(1, keepdim=True)[1]
            # 실제값(0,1,..,9), 위에서 추출한 인덱스정보(0,1, .. 9) => eq() => sum() => item() => 누적합
            # shape 맞춰주는 함수 : 원본.view_as(바꾸고싶은대상) => 차원값을 모두 곱한값이 동일해야함
            # 64개 데이터중에 원본과 일치하는 개수 획득
            mean_acc += y_pred.eq( target.view_as(y_pred) ).sum().item()
            # 맞춘개수를 계소 누적합

    # 4. mean_loss, mean_acc(맞춘개수/전체개수*100) 계산
    mean_loss = mean_loss / len(test_loader.dataset)
    mean_acc  = mean_acc / len(test_loader.dataset) * 100 # 퍼센트 단위환산

    # 5. mean_loss, mean_acc 반환
    return mean_loss, mean_acc

- 특정 모듈 단위 테스트

In [12]:
a = torch.Tensor( [1, 1, 2, 0 ] )
b = torch.Tensor( [0, 0, 1, 1 ] )
print( a, b )

print( F.cross_entropy( a, b) ) # 결과값을 평균내여서 리턴 기본값, mean
print( F.cross_entropy( a, b, reduction='sum' ).item() ) # 평균 X, 합산으로 기록

tensor([1., 1., 2., 0.]) tensor([0., 0., 1., 1.])
tensor(3.2530)
3.253046989440918


In [13]:
tenser = torch.arange(16).view(4,4)
'''
tensor([[ 0,  1,  2,  3],  <- 샘플 데이터 1
        [ 4,  5,  6,  7],  <- 샘플 데이터 2
        [ 8,  9, 10, 11],  <- 샘플 데이터 3
        [12, 13, 14, 15]]) <- 샘플 데이터 4 라고 가정하면 이해하기 쉬움
'''
# 원본
print( tenser )
# 원본 테서들중 가장 큰값
print( tenser.max() )
# 텐서의 1=> 두번재차원, 차원번호 1번에 최대값추출 => [값, 인덱스]
print( tenser.max(1) )
# 텐서의 1=> 두번재차원, 차원번호 1번에 최대값추출 => 인덱스만 추출
print( tenser.max(1)[1] )
# 차원을 유지하면서 최대값을 가진 인덱스만 추출
print( tenser.max(1, keepdim=True)[1] )

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
tensor(15)
torch.return_types.max(
values=tensor([ 3,  7, 11, 15]),
indices=tensor([3, 3, 3, 3]))
tensor([3, 3, 3, 3])
tensor([[3],
        [3],
        [3],
        [3]])


## 훈련 진행

In [14]:
%%time
EPOCHS = 10
# 학습 성과에 대한 시각화
losses, accs = list(), list()
for epoch in range( 1, EPOCHS+1):
    # 훈련
    train( model, train_loader, opimzer, epoch)
    # 테스트
    mean_loss, mean_acc = test( model, test_loader)
    losses.append( mean_loss)
    accs.append(   mean_acc)
    # 로그 출력
    print( f'epoch:{epoch}\t loss={mean_loss}\t acc={mean_acc}' )

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch:1	 mini-batch:0	 loss:2.302290201187134
Epoch:1	 mini-batch:200	 loss:0.6654482483863831
Epoch:1	 mini-batch:400	 loss:0.583935022354126
Epoch:1	 mini-batch:600	 loss:0.6952916383743286
Epoch:1	 mini-batch:800	 loss:0.3591659963130951
epoch:1	 loss=0.4454143620491028	 acc=83.91
Epoch:2	 mini-batch:0	 loss:0.4541183114051819
Epoch:2	 mini-batch:200	 loss:0.23918388783931732
Epoch:2	 mini-batch:400	 loss:0.23563280701637268
Epoch:2	 mini-batch:600	 loss:0.5074765682220459
Epoch:2	 mini-batch:800	 loss:0.42524269223213196
epoch:2	 loss=0.3861288519382477	 acc=86.1
Epoch:3	 mini-batch:0	 loss:0.2548559308052063
Epoch:3	 mini-batch:200	 loss:0.26245588064193726
Epoch:3	 mini-batch:400	 loss:0.30270087718963623
Epoch:3	 mini-batch:600	 loss:0.48542168736457825
Epoch:3	 mini-batch:800	 loss:0.42018911242485046
epoch:3	 loss=0.3342477601051331	 acc=87.71
Epoch:4	 mini-batch:0	 loss:0.2687333822250366
Epoch:4	 mini-batch:200	 loss:0.2867429852485657
Epoch:4	 mini-batch:400	 loss:0.4518964

# 학습 결과 정리

In [15]:
# 아래 데이터를 기반으로 시각화 가능 => QnA
losses, accs

([0.4454143620491028,
  0.3861288519382477,
  0.3342477601051331,
  0.3165507873535156,
  0.2940669124603271,
  0.289024803686142,
  0.283906627869606,
  0.25476895473003386,
  0.25680886676311493,
  0.25058667533397677],
 [83.91,
  86.1,
  87.71,
  88.44,
  89.56,
  89.73,
  89.47,
  90.63,
  90.53,
  91.21000000000001])

- cuda 기반 학습 진행
    - 10 epoch 기준
        - loss=0.25058667533397677
        - acc=91.21000000000001
    - 추가 학습 더 필요 (학습 횟수가 적음)
        - 모델 성능이 지속적으로 향상됨이 보임
        - 필요시 조기 학습 종료 추가

# 모델 덤플 및 로드

- 기본 모델만 저장

In [16]:
# 최종 모델 저장
torch.save( model, 'model.pt' )

In [17]:
# 로드
# map_location : cpu or cuda 설정 맞춤
model2 = torch.load( 'model.pt', map_location=DEVICE)

In [18]:
# 테스트
loss, acc = (0,0)
model2.eval()
with torch.no_grad():
    for idx, (data, target) in enumerate( test_loader ):
        data, target    = data.to(DEVICE), target.to(DEVICE)
        output          = model( data )
        loss           += F.cross_entropy( output, target, reduction='sum' ).item()
        y_pred          = output.max(1, keepdim=True)[1]
        acc            += y_pred.eq( target.view_as(y_pred) ).sum().item()
        # 1회만 진행
        print( loss/BATCH_SIZE, acc/BATCH_SIZE*100)
        break

0.23934680223464966 93.75


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


- model.state_dict() 사용

In [19]:
# 모델 상태 저장 하기
torch.save( model.state_dict(), 'model_state_dict.pt')

In [20]:
# 상태 로드
state = torch.load( 'model_state_dict.pt', map_location=DEVICE)
# 기존 모델에 교체(설정)
model2.load_state_dict( state )

<All keys matched successfully>

In [21]:
loss, acc = (0,0)
model2.eval()
with torch.no_grad():
    for idx, (data, target) in enumerate( test_loader ):
        data, target    = data.to(DEVICE), target.to(DEVICE)
        output          = model( data )
        loss           += F.cross_entropy( output, target, reduction='sum' ).item()
        y_pred          = output.max(1, keepdim=True)[1]
        acc            += y_pred.eq( target.view_as(y_pred) ).sum().item()
        # 1회만 진행
        print( loss/BATCH_SIZE, acc/BATCH_SIZE*100)
        break

# 공급데이터가 변경되어서 손실값, 정확도가 상이해짐

0.15921537578105927 93.75


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
